In [3]:
push!(LOAD_PATH, "/home/amir/Work/gdmrg")
import Tmp

LoadError: LoadError: LoadError: [91msyntax: "<" is not a unary operator[39m
while loading /home/amir/Work/gdmrg/MPS.jl, in expression starting on line 277
while loading /home/amir/Work/gdmrg/Tmp.jl, in expression starting on line 8

In [58]:
reload("Tmp")
include("test.jl")

LoadError: LoadError: LoadError: [91msyntax: "<" is not a unary operator[39m
while loading /home/amir/Work/gdmrg/MPO.jl, in expression starting on line 13
while loading /home/amir/Work/gdmrg/Tmp.jl, in expression starting on line 8

In [ ]:
Tmp.runstuff(8,100)

In [24]:
singlet = Vector{Complex128}([0, 1, -1, 0])
ketstate = kron(singlet, singlet)
mps = Tmp.MPS(4, 2, ketstate)
mpo = Tmp.MPO(4, 2)
Tmp.measure(mps, mpo)

LoadError: [91mUndefVarError: expectation_mpo not defined[39m

In [57]:
H = Tmp.heisenberg(4)
e, v = eigs(H, nev=1, which="SR")
@show e, v
mps = Tmp.MPS(4, 2, v[:,1])
mpo = Tmp.MPO{Float64}(4, 2)
Tmp.measure(mps, mpo)

(e, v) = ([-1.61603], [0.0; 2.08167e-17; 1.11022e-16; -0.149429; -1.11022e-16; 0.557678; -0.408248; 1.11022e-16; -1.11022e-16; -0.408248; 0.557678; -5.55112e-17; -0.149429; 1.38778e-17; -1.11022e-16; 5.55112e-17])


LoadError: [91mMethodError: no method matching Tmp.MPO{Float64}(::Int64, ::Int64)[39m